In [168]:
from dotenv import load_dotenv
import os
load_dotenv()
from sqlalchemy import create_engine
import pandas as pd

from time import sleep

In [169]:
import os
print(os.getcwd())

/home/arman/arman/code/vip/tractor/apps/wiki-tajrobeh


In [170]:
DATABASE_URL = os.getenv("DATABASE_URL")

In [171]:
engine = create_engine(DATABASE_URL)

In [172]:
df = pd.read_sql_query('SELECT * FROM "Skills"', engine)

In [173]:
skills = df["SkillTitle"].to_list()

In [174]:
skills

['ندارد',
 'مکانیکی',
 'تعمیرات سیستم هیدرولیک',
 'برق و باطری سازی',
 'نصب کیت بدنه خودرو',
 'تودوزی و تعمیرات صندلی',
 'توربو شارژ',
 'فروشگاه قطعات بدنه خودرو',
 'نمایندگی های مجاز',
 'اجاره خودرو',
 'ارتقاء سیستم صوتی و تصویری',
 'حمل خودرو با یدک کش',
 'حمل خودرو با خودرو بر',
 'تعمیرات رینگ',
 'تعویض ترموستات خودرو',
 'تعویض قطعات بدنه',
 'تعویض تسمه خودرو',
 'خدمات شیشه خودرو',
 'نمایشگاه\u200c خودرو',
 'تیونینگ و آپشن',
 'خدمات آینه اتومبیل',
 'کارواش',
 'لوازم یدکی خودرو',
 'صافکاری و نقاشی',
 'رینگ و لاستیک',
 'رادیاتورسازی',
 'کارشناسی خودرو',
 'تعمیرات موتور خودرو',
 'سرویس های دوره ای',
 'حمل خودرو',
 'سوخت رسانی در محل',
 'فروشگاه قطعات یدکی استوک',
 'لوازم یدکی ماشین سنگین',
 'فروشگاه لوازم یدکی موتور',
 'تعمیرات گیربکس',
 'تعمیر سیستم تعلیق و جلوبندی',
 'تعمیرات ترمز خودرو',
 'تعمیرات دیفرانسیل',
 'تعمیرات پلوس و شفت',
 'خدمات باتری ماشین',
 'تعمیر دینام خودرو',
 'تعویض کویل و وایرها',
 'تعمیرات سیم کشی خودرو',
 'تعمیر و تعویض فیوزها',
 'تعمیر سنسورهای خودرو',
 'تعمیر ب

In [175]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import os
import json

# OpenRouter Configuration
api_key = os.getenv("OPENROUTER_API_KEY")
if not api_key:
    print("Warning: OPENROUTER_API_KEY not found in environment variables.")

llm = ChatOpenAI(
    model="google/gemini-2.0-flash-001",
    # model="openai/gpt-5-nano",
    # model="openai/gpt-oss-120b",
    # model="x-ai/grok-4-fast",
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1",
)

In [176]:
template = """
You are an expert at categorizing vendors into predefined skill categories from their business titles.

Valid Skills List:
{skills_list}

Input Vendor Name:
{vendor_name}

Find the most relevant skill from the "Valid Skills List" for the provided vendor name.
Return ONLY the exact string of the nearest skill from the list. Do not include any other text or explanation.Do not make up skills.

Nearest Skill:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["skills_list", "vendor_name"],
)

async def get_nearest_skill(vendor_name, skills):
    # Format skills as a list
    skills_str = "\n".join([f"- {s}" for s in skills])
    
    # Format prompt
    formatted_prompt = prompt.format(
        skills_list=skills_str,
        vendor_name=vendor_name
    )
    
    # Call the LLM using ainvoke for async compatibility
    response = await llm.ainvoke(formatted_prompt)
    
    return response.content.strip()

# Example usage
test_vendor = "تعمیرگاه تخصصی رنو"
result = await get_nearest_skill(test_vendor, skills)
result

'تعمیرات موتور خودرو'

In [177]:
# Use the relative path to the data folder
import os

# Check possible locations for the excel file
excel_path = "apps/wiki-tajrobeh/data/wiki-tajrobeh.xlsx"
if not os.path.exists(excel_path):
    excel_path = "data/wiki-tajrobeh.xlsx"

print(f"Target Excel file: {excel_path}")

try:
    vendors_df = pd.read_excel(excel_path)
    vendors_titles = vendors_df["name"].dropna().unique().tolist()
    print(f"Loaded {len(vendors_titles)} unique vendor titles.")
except Exception as e:
    print(f"Error loading Excel: {e}")
    vendors_df = pd.DataFrame() # Initialize as empty to avoid NameError
    vendors_titles = []

Target Excel file: data/wiki-tajrobeh.xlsx
Loaded 17585 unique vendor titles.


In [178]:
# # Update all records in the DataFrame
# # Note: This will process all rows one by one and save after each update.

# for i in vendors_df.index:
#     title = vendors_df.at[i, "name"]
    
#     if pd.isna(title):
#         continue
        
#     try:
#         # Get the nearest skill from the LLM
#         sleep(0.1)  # Add a delay to avoid rate limiting
#         skill = await get_nearest_skill(title, skills)
        
#         # Overwrite the category_names column
#         vendors_df.at[i, "category_names"] = skill
        
#         # Save to Excel immediately after update
#         vendors_df.to_excel(excel_path, index=False)
        
#         print(f"[{i+1}/{len(vendors_df)}] Updated and Saved: '{title}' -> '{skill}'")
#     except Exception as e:
#         print(f"❌ Error at index {i} ('{title}'): {e}")

# # Display the first few updated rows
# vendors_df.head()

In [180]:
# Alternative Approach: Update all records in memory first, then save ONCE at the end
# Use this block if you prefer speed and don't need intermediate saves.

print("Starting bulk processing...")
updated_count = 0

for i in vendors_df.index:
    title = vendors_df.at[i, "name"]
    
    if pd.isna(title):
        continue
        
    try:
        # Get the nearest skill from the LLM
        sleep(0.1)  # Add a delay to avoid rate limiting
        skill = await get_nearest_skill(title, skills)
        
        # Update the DataFrame in memory
        vendors_df.at[i, "category_names"] = skill
        
        updated_count += 1
        print(f"[{i+1}/{len(vendors_df)}] Processed: '{title}' -> '{skill}'")
    except Exception as e:
        print(f"❌ Error at index {i} ('{title}'): {e}")

# Save to Excel once after all processing is done
print(f"Saving {updated_count} updates to {excel_path}...")
vendors_df.to_excel(excel_path, index=False)
print("Bulk update complete and saved.")

Starting bulk processing...
[1/17744] Processed: 'تعمیرگاه مکانیکی رضا برزگر در آذرشهر' -> 'مکانیکی'
[2/17744] Processed: 'تعمیر گاه مکانیکی قائم در آذرشهر' -> 'مکانیکی'
[3/17744] Processed: 'مکانیکی وتست موتوراحمد در آذرشهر' -> 'تعمیرات موتور خودرو'
[4/17744] Processed: 'مکانیکی و آهنگری ام آی تیونینگ در آذرشهر' -> 'مکانیکی'
[5/17744] Processed: 'مکانیکی حسین نژاد در آذرشهر' -> 'مکانیکی'
[6/17744] Processed: 'تعمیرگاه اهنگری اسد رجبی در آذرشهر' -> 'ندارد'
[7/17744] Processed: 'تعمیرگاه سهند در آذرشهر' -> 'مکانیکی'
[8/17744] Processed: 'باطریسازی امید در اهر' -> 'برق و باطری سازی'
[9/17744] Processed: 'مکانیکی افسر پژو در اهر' -> 'مکانیکی'
[10/17744] Processed: 'مکانیکی محرم بخشیان در اهر' -> 'مکانیکی'
[11/17744] Processed: 'تعمیرگاه آهنگری محسن حسن پور در اهر' -> 'مکانیکی'
[12/17744] Processed: 'تعمیرگاه پژو ارسباران در اهر' -> 'تعمیرات موتور خودرو'
[13/17744] Processed: 'صافکاری سنتی و PDR امیر در اهر' -> 'صافکاری pdr'
[14/17744] Processed: 'تعمیرگاه ایرانیان در اهر' -> 'مکانیکی'
[15

In [182]:
# Save the updated data back to the original Excel file
print(f"Saving updated data to {excel_path}...")
vendors_df.to_excel(excel_path, index=False)
print(f"Successfully updated {excel_path}")

Saving updated data to data/wiki-tajrobeh.xlsx...
Successfully updated data/wiki-tajrobeh.xlsx
